In [1]:
# 경고무시
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
#00. 패키지 호출
import pandas as pd 
import numpy as np 
#from xgboost import XGBRegressor # conda install py-xgboost
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

In [161]:
#01. 데이터셋 불러오기. 
df_문어_raw = pd.read_csv("C:/ken/feature-target/문어.csv")
df_문어_raw.head()

,date,?�균기온,최�?기온,최고기온,?�강?�량,?�균?�슬?�온??최소?��??�도,?�균?��??�도,?�계?�조?�간,?�계?�사???�균지면온???�계?�?�증발량,?�계?�형증발??문어,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,2018-01-01T00:00:00Z,-1.3,-5.1,3.8,0.0,-13.9,19,39.1,8.3,6.14,-1.7,1.1,1.6,50.333978
1,2018-01-02T00:00:00Z,-1.8,-4.3,1.8,0.0,-14.1,19,42.0,7.9,5.36,-2.3,1.3,1.9,57.135783
2,2018-01-03T00:00:00Z,-4.7,-7.1,-0.4,0.0,-15.8,28,42.3,8.6,6.56,-3.4,1.0,1.5,52.790193
3,2018-01-04T00:00:00Z,-4.7,-8.7,-0.7,0.0,-15.6,31,43.0,6.2,4.73,-3.3,0.8,1.1,60.221465
4,2018-01-05T00:00:00Z,-3.0,-5.6,1.6,0.0,-12.5,32,48.4,8.2,5.96,-2.6,0.9,1.3,58.932383


In [162]:
#02. 날짜 제거하기. 
df_문어 = df_문어_raw
del df_문어['date']
df_문어

,?�균기온,최�?기온,최고기온,?�강?�량,?�균?�슬?�온??최소?��??�도,?�균?��??�도,?�계?�조?�간,?�계?�사???�균지면온???�계?�?�증발량,?�계?�형증발??문어,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,-1.3,-5.1,3.8,0.0,-13.9,19,39.1,8.3,6.14,-1.7,1.1,1.6,50.333978
1,-1.8,-4.3,1.8,0.0,-14.1,19,42.0,7.9,5.36,-2.3,1.3,1.9,57.135783
2,-4.7,-7.1,-0.4,0.0,-15.8,28,42.3,8.6,6.56,-3.4,1.0,1.5,52.790193
3,-4.7,-8.7,-0.7,0.0,-15.6,31,43.0,6.2,4.73,-3.3,0.8,1.1,60.221465
4,-3.0,-5.6,1.6,0.0,-12.5,32,48.4,8.2,5.96,-2.6,0.9,1.3,58.932383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,-1.7,-4.6,2.6,0.0,-12.4,23,45.4,9.0,10.36,-0.9,1.5,2.2,46.606606
726,1.1,-3.6,6.1,0.0,-9.0,34,47.6,5.8,8.09,-0.7,1.4,2.0,48.402502
727,3.8,1.1,6.2,1.4,-5.4,37,52.5,0.0,3.28,1.7,1.1,1.6,50.622953
728,2.7,-5.7,6.8,0.4,-2.3,43,70.3,0.0,2.24,3.3,0.9,1.3,50.187176


In [163]:
#03. 데이터셋 스케일링하기. 
scaler = StandardScaler()
df_문어_scaling = scaler.fit_transform(df_문어.iloc[:,:-1])
df_문어_scaling = pd.DataFrame(df_문어_scaling, columns=['평균기온','최저기온','최고기온','일강수량','평균이슬점온도','최소상대습도',	'평균상대습도','합계일조시간','합계일사량','평균지면온도','합계대형증발량','합계소형증발량'])

df_문어_scaling['문어'] = df_문어.iloc[:,-1].round().astype('int')
df_문어_scaling

,평균기온,최저기온,최고기온,일강수량,평균이슬점온도,최소상대습도,평균상대습도,합계일조시간,합계일사량,평균지면온도,합계대형증발량,합계소형증발량,문어
0,-1.345873,-1.296793,-1.316587,-0.290113,-1.449840,-1.087268,-1.183451,0.322310,-1.072483,-1.405386,-1.062767,-1.052238,50
1,-1.391963,-1.223715,-1.498441,-0.290113,-1.465927,-1.087268,-0.993211,0.221771,-1.181541,-1.455957,-0.938556,-0.921795,57
2,-1.659281,-1.479489,-1.698481,-0.290113,-1.602662,-0.479323,-0.973531,0.397714,-1.013759,-1.548669,-1.124872,-1.095719,53
3,-1.659281,-1.625646,-1.725759,-0.290113,-1.586575,-0.276675,-0.927611,-0.205519,-1.269627,-1.540241,-1.249083,-1.269643,60
4,-1.502577,-1.342467,-1.516626,-0.290113,-1.337235,-0.209125,-0.573371,0.297175,-1.097650,-1.481242,-1.186977,-1.182681,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,-1.382745,-1.251119,-1.425699,-0.290113,-1.329192,-0.817070,-0.770171,0.498252,-0.482447,-1.337959,-0.814345,-0.791353,47
726,-1.124644,-1.159771,-1.107455,-0.290113,-1.055722,-0.074027,-0.625851,-0.306058,-0.799836,-1.321102,-0.876451,-0.878315,48
727,-0.875762,-0.730435,-1.098362,-0.153897,-0.766166,0.128621,-0.304411,-1.763871,-1.472364,-1.118820,-1.062767,-1.052238,51
728,-0.977158,-1.351602,-1.043806,-0.251194,-0.516827,0.533918,0.863269,-1.763871,-1.617776,-0.983966,-1.186977,-1.182681,50


In [164]:
# 결측값 확인
df_문어_scaling.isna().sum()

평균기온       0
최저기온       0
최고기온       0
일강수량       0
평균이슬점온도    0
최소상대습도     0
평균상대습도     0
합계일조시간     0
합계일사량      0
평균지면온도     0
합계대형증발량    0
합계소형증발량    0
문어         0
dtype: int64

In [165]:
#04. 데이터셋 탐색하기. 
df_문어_scaling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   평균기온     730 non-null    float64
 1   최저기온     730 non-null    float64
 2   최고기온     730 non-null    float64
 3   일강수량     730 non-null    float64
 4   평균이슬점온도  730 non-null    float64
 5   최소상대습도   730 non-null    float64
 6   평균상대습도   730 non-null    float64
 7   합계일조시간   730 non-null    float64
 8   합계일사량    730 non-null    float64
 9   평균지면온도   730 non-null    float64
 10  합계대형증발량  730 non-null    float64
 11  합계소형증발량  730 non-null    float64
 12  문어       730 non-null    int32  
dtypes: float64(12), int32(1)
memory usage: 71.4 KB


In [166]:
df_문어_scaling.describe()

,평균기온,최저기온,최고기온,일강수량,평균이슬점온도,최소상대습도,평균상대습도,합계일조시간,합계일사량,평균지면온도,합계대형증발량,합계소형증발량,문어
count,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,7.300000e+02,730.000000
mean,1.946692e-17,9.733462e-17,8.760116e-17,7.300097e-17,9.733462e-18,2.044027e-16,-2.530700e-16,1.496520e-16,-4.866731e-17,4.866731e-17,2.238696e-16,1.703356e-16,60.000000
std,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,1.000686e+00,9.543058
min,-2.590287e+00,-2.456914e+00,-2.635030e+00,-2.901125e-01,-2.551762e+00,-1.830311e+00,-2.246172e+00,-1.763871e+00,-1.930970e+00,-2.045946e+00,-1.559609e+00,-1.574009e+00,40.000000
25%,-8.273677e-01,-8.035132e-01,-8.414928e-01,-2.901125e-01,-7.802421e-01,-7.495206e-01,-7.193312e-01,-9.281420e-01,-7.592886e-01,-9.418235e-01,-8.764507e-01,-8.783145e-01,53.000000
50%,9.672458e-02,2.318741e-02,1.473392e-01,-2.901125e-01,8.641506e-02,-1.415761e-01,-4.857097e-02,2.971748e-01,-8.955678e-02,2.159060e-03,-1.932922e-01,-1.826199e-01,59.000000
75%,8.733308e-01,8.864273e-01,8.361119e-01,-2.633559e-01,8.324239e-01,5.339178e-01,6.402293e-01,7.244648e-01,7.745231e-01,8.618574e-01,7.382876e-01,7.304794e-01,66.000000
max,1.880384e+00,1.936931e+00,1.938603e+00,9.099033e+00,1.678973e+00,3.776288e+00,2.614790e+00,1.654449e+00,2.151738e+00,2.117691e+00,2.663553e+00,2.643640e+00,93.000000


In [167]:
#05. train, test 데이터셋으로 분할하기. 
X_train, X_test, y_train, y_test = train_test_split(
    df_문어_scaling.iloc[:,:-1], df_문어_scaling.iloc[:,-1], test_size=0.5 
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((365, 12), (365, 12), (365,), (365,))

In [168]:
#06. svm 모델로 머신러닝하기. 
#(1) 모형 생성하기. 
svc = SVC(random_state=2021)

#(2) 하이퍼 파라미터 확인하기. 
svc.get_params

<bound method BaseEstimator.get_params of SVC(random_state=2021)>

In [169]:
#(3) GridSearch를 위한 하이퍼파라미터 값 지정하기. (recycle)
params = {'C':[ 0.05, 1, 5, 10,50,60,65, 70, 75 ,80,85,90, 100,300],
'gamma':[0.01,0.02, 0.03, 0.05, 1, 3, 7, 15,50,90,100]
}


#(4) GridsearchCV() 메서드를 이용해 훈련 모델 할당하기. 

gscv_svc = GridSearchCV(svc, param_grid=params, cv =5, scoring = 'accuracy')

#(5) 모형 학습하기. 
gscv_svc.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVC(random_state=2021),
             param_grid={'C': [0.05, 1, 5, 10, 50, 60, 65, 70, 75, 80, 85, 90,
                               100, 300],
                         'gamma': [0.01, 0.02, 0.03, 0.05, 1, 3, 7, 15, 50, 90,
                                   100]},
             scoring='accuracy')

In [170]:
#(6) 최적 하이퍼 파라미터 확인하기. 
gscv_svc.best_params_

{'C': 1, 'gamma': 0.01}

In [171]:
#(7) 최적 파라미터에 대한 평가 점수 확인하기. 
gscv_svc.best_score_

0.07397260273972603

In [172]:
#(8) test 데이터셋으로 모형 예측 및 평가하기. 
gscv_svc_fit = gscv_svc.best_estimator_
svc_pred = gscv_svc_fit.predict(X_test)
r2_score(y_test,svc_pred)

0.2180576907745071